<a href="https://colab.research.google.com/github/Driante/fantacalcio/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import io, os
from collections import deque 
from src.calendario import *
from itertools import permutations

In [5]:
calendar_raw = import_calendar()
df_persquadra,ordine_giornate = clean_calendar(calendar_raw)
squadre =df_persquadra.index.values

In [40]:
def crea_classifica(df_persquadra):
    ngiornate = (df_persquadra.loc[df_persquadra.index[0]]['Fantapunti'] ==0).argmax()
    nsquadre = df_persquadra.shape[0]
    df_persquadra['Punti'] = [np.zeros(ngiornate) for _ in range(nsquadre)]
    for giornata in range(ngiornate):
        for squadra in range(nsquadre):
            g1 = df_persquadra['Gol'][squadra][giornata]
            g2 = df_persquadra.loc[df_persquadra['Avversari'][squadra][giornata]]['Gol'][giornata]
            if g1> g2:
                df_persquadra['Punti'][squadra][giornata] += 3
            elif g1==g2:
                df_persquadra['Punti'][squadra][giornata] += 1
    return 

def crea_calendario_template(nsquadre):
    #Crea un template di calendario di girone all'italiana
    squadre_template = ['A','B','C','D','E','F','G','H']
    calendario_template = []
    lista_avversari=[[] for _ in range(nsquadre)]
    for shift in range(nsquadre-1):
        temp = deque(squadre_template[1:])
        temp.rotate(shift)
        squadre_template_rot = [squadre_template[0]] + list(temp)
        giornata = []

        for n in range(int(nsquadre/2)):
            giornata.append(squadre_template_rot[n] + '-' + squadre_template_rot[-n-1])
            lista_avversari[squadre_template.index(squadre_template_rot[n])].append(squadre_template_rot[-n-1])
            lista_avversari[squadre_template.index(squadre_template_rot[-n-1])].append(squadre_template_rot[n])
        calendario_template.append(giornata)
    return squadre_template,calendario_template,lista_avversari
def tutti_calendari(squadre):
    #permuta tutti i possibli assegnamenti delle squadre al template, generando tutti i possibili calendari 
    #sotto forma di liste di avversari
    nsquadre = len(squadre)
    squadre_template,calendario_template,lista_avversari = crea_calendario_template(nsquadre)
    cDict_template = {squadre_template[n] : lista_avversari[n] for n in range(nsquadre)}
    all_calendars = []
    i= 0
    for squadre_perm in permutations(squadre):
        cDictnew = {}
        template_matching = {squadre_template[n] : squadre_perm[n] for n in range(nsquadre)}
        for n in range(nsquadre):
            cDictnew[squadre_perm[n]] = [template_matching.get(a,'default') for a in cDict_template[squadre_template[n]]]*7
        all_calendars.append(cDictnew)
        i+=1
    return all_calendars

In [41]:
all_calendars = tutti_calendari(squadre)

In [43]:
df_persquadra_mut = df_persquadra.copy()

In [44]:
classifiche = [];
ncal = 0
for cal in all_calendars:
    for s in cal:
        df_persquadra_mut.loc[s]['Avversari'] = cal[s]
    crea_classifica(df_persquadra_mut)
    classifiche.append(df_persquadra_mut['Punti'].apply(np.sum).sort_values(ascending=False))
    print(ncal, "Capolista: ",classifiche[ncal].index[0])
    ncal +=1

0 Capolista:  Capitan Martella
1 Capolista:  Paris FC
2 Capolista:  CACCOLA F.C.
3 Capolista:  CACCOLA F.C.
4 Capolista:  CACCOLA F.C.
5 Capolista:  CACCOLA F.C.
6 Capolista:  CACCOLA F.C.
7 Capolista:  Paris FC
8 Capolista:  Capitan Martella
9 Capolista:  CACCOLA F.C.
10 Capolista:  Capitan Martella
11 Capolista:  CACCOLA F.C.
12 Capolista:  CACCOLA F.C.
13 Capolista:  CACCOLA F.C.
14 Capolista:  Capitan Martella
15 Capolista:  SamPDORRR FC
16 Capolista:  CACCOLA F.C.
17 Capolista:  CACCOLA F.C.
18 Capolista:  CACCOLA F.C.
19 Capolista:  CACCOLA F.C.
20 Capolista:  Paris FC
21 Capolista:  Capitan Martella
22 Capolista:  CACCOLA F.C.
23 Capolista:  CACCOLA F.C.
24 Capolista:  Capitan Martella
25 Capolista:  Capitan Martella
26 Capolista:  CACCOLA F.C.
27 Capolista:  SamPDORRR FC
28 Capolista:  CACCOLA F.C.
29 Capolista:  Capitan Martella
30 Capolista:  Capitan Martella
31 Capolista:  Capitan Martella
32 Capolista:  Capitan Martella
33 Capolista:  Capitan Martella
34 Capolista:  Capitan

280 Capolista:  CACCOLA F.C.
281 Capolista:  CACCOLA F.C.
282 Capolista:  CACCOLA F.C.
283 Capolista:  CACCOLA F.C.
284 Capolista:  Paris FC
285 Capolista:  Capitan Martella
286 Capolista:  CACCOLA F.C.
287 Capolista:  CACCOLA F.C.
288 Capolista:  CACCOLA F.C.
289 Capolista:  CACCOLA F.C.
290 Capolista:  Capitan Martella
291 Capolista:  CACCOLA F.C.
292 Capolista:  Capitan Martella
293 Capolista:  Capitan Martella
294 Capolista:  Capitan Martella
295 Capolista:  Capitan Martella
296 Capolista:  CACCOLA F.C.
297 Capolista:  Capitan Martella
298 Capolista:  CACCOLA F.C.
299 Capolista:  Paris FC
300 Capolista:  Capitan Martella
301 Capolista:  CACCOLA F.C.
302 Capolista:  CACCOLA F.C.
303 Capolista:  SamPDORRR FC
304 Capolista:  CACCOLA F.C.
305 Capolista:  Paris FC
306 Capolista:  Capitan Martella
307 Capolista:  Capitan Martella
308 Capolista:  CACCOLA F.C.
309 Capolista:  SamPDORRR FC
310 Capolista:  Civediamoamaggio 
311 Capolista:  Capitan Martella
312 Capolista:  Paris FC
313 Capoli

561 Capolista:  Paris FC
562 Capolista:  Civediamoamaggio 
563 Capolista:  Capitan Martella
564 Capolista:  Capitan Martella
565 Capolista:  SamPDORRR FC
566 Capolista:  Capitan Martella
567 Capolista:  Capitan Martella
568 Capolista:  Civediamoamaggio 
569 Capolista:  Capitan Martella
570 Capolista:  Paris FC
571 Capolista:  Capitan Martella
572 Capolista:  CACCOLA F.C.
573 Capolista:  Capitan Martella
574 Capolista:  Civediamoamaggio 
575 Capolista:  Paris FC
576 Capolista:  CACCOLA F.C.
577 Capolista:  Paris FC
578 Capolista:  Paris FC
579 Capolista:  CACCOLA F.C.
580 Capolista:  Paris FC
581 Capolista:  Capitan Martella
582 Capolista:  Capitan Martella
583 Capolista:  CACCOLA F.C.
584 Capolista:  Civediamoamaggio 
585 Capolista:  CACCOLA F.C.
586 Capolista:  Civediamoamaggio 
587 Capolista:  Capitan Martella
588 Capolista:  Paris FC
589 Capolista:  CACCOLA F.C.
590 Capolista:  Civediamoamaggio 
591 Capolista:  CACCOLA F.C.
592 Capolista:  Civediamoamaggio 
593 Capolista:  Capitan M

842 Capolista:  CACCOLA F.C.
843 Capolista:  Capitan Martella
844 Capolista:  CACCOLA F.C.
845 Capolista:  Paris FC
846 Capolista:  CACCOLA F.C.
847 Capolista:  CACCOLA F.C.
848 Capolista:  Capitan Martella
849 Capolista:  CACCOLA F.C.
850 Capolista:  Capitan Martella
851 Capolista:  CACCOLA F.C.
852 Capolista:  CACCOLA F.C.
853 Capolista:  CACCOLA F.C.
854 Capolista:  Capitan Martella
855 Capolista:  asdMalerba
856 Capolista:  Capitan Martella
857 Capolista:  CACCOLA F.C.
858 Capolista:  CACCOLA F.C.
859 Capolista:  SamPDORRR FC
860 Capolista:  Capitan Martella
861 Capolista:  SamPDORRR FC
862 Capolista:  Capitan Martella
863 Capolista:  CACCOLA F.C.
864 Capolista:  Capitan Martella
865 Capolista:  Capitan Martella
866 Capolista:  CACCOLA F.C.
867 Capolista:  CACCOLA F.C.
868 Capolista:  Capitan Martella
869 Capolista:  Paris FC
870 Capolista:  Paris FC
871 Capolista:  Capitan Martella
872 Capolista:  Capitan Martella
873 Capolista:  Capitan Martella
874 Capolista:  Capitan Martella
8

1118 Capolista:  Capitan Martella
1119 Capolista:  Capitan Martella
1120 Capolista:  Paris FC
1121 Capolista:  Capitan Martella
1122 Capolista:  Capitan Martella
1123 Capolista:  Paris FC
1124 Capolista:  Capitan Martella
1125 Capolista:  Capitan Martella
1126 Capolista:  Paris FC
1127 Capolista:  Capitan Martella
1128 Capolista:  Capitan Martella
1129 Capolista:  Capitan Martella
1130 Capolista:  Capitan Martella
1131 Capolista:  Capitan Martella
1132 Capolista:  Capitan Martella
1133 Capolista:  Capitan Martella
1134 Capolista:  Paris FC
1135 Capolista:  SamPDORRR FC
1136 Capolista:  Gargantua 
1137 Capolista:  SamPDORRR FC
1138 Capolista:  Capitan Martella
1139 Capolista:  Paris FC
1140 Capolista:  Paris FC
1141 Capolista:  SamPDORRR FC
1142 Capolista:  Paris FC
1143 Capolista:  SamPDORRR FC
1144 Capolista:  Capitan Martella
1145 Capolista:  Paris FC
1146 Capolista:  Paris FC
1147 Capolista:  Paris FC
1148 Capolista:  Capitan Martella
1149 Capolista:  Capitan Martella
1150 Capolista

...

39226 Capolista:  Capitan Martella
39227 Capolista:  Capitan Martella
39228 Capolista:  Capitan Martella
39229 Capolista:  Capitan Martella
39230 Capolista:  Capitan Martella
39231 Capolista:  Capitan Martella
39232 Capolista:  SamPDORRR FC
39233 Capolista:  Capitan Martella
39234 Capolista:  Paris FC
39235 Capolista:  Capitan Martella
39236 Capolista:  Capitan Martella
39237 Capolista:  Capitan Martella
39238 Capolista:  Capitan Martella
39239 Capolista:  Capitan Martella
39240 Capolista:  Capitan Martella
39241 Capolista:  Capitan Martella
39242 Capolista:  CACCOLA F.C.
39243 Capolista:  CACCOLA F.C.
39244 Capolista:  CACCOLA F.C.
39245 Capolista:  Capitan Martella
39246 Capolista:  Capitan Martella
39247 Capolista:  Capitan Martella
39248 Capolista:  CACCOLA F.C.
39249 Capolista:  CACCOLA F.C.
39250 Capolista:  CACCOLA F.C.
39251 Capolista:  CACCOLA F.C.
39252 Capolista:  CACCOLA F.C.
39253 Capolista:  Capitan Martella
39254 Capolista:  CACCOLA F.C.
39255 Capolista:  CACCOLA F.C.
39

39490 Capolista:  CACCOLA F.C.
39491 Capolista:  Paris FC
39492 Capolista:  Capitan Martella
39493 Capolista:  Capitan Martella
39494 Capolista:  CACCOLA F.C.
39495 Capolista:  Capitan Martella
39496 Capolista:  CACCOLA F.C.
39497 Capolista:  CACCOLA F.C.
39498 Capolista:  CACCOLA F.C.
39499 Capolista:  Paris FC
39500 Capolista:  CACCOLA F.C.
39501 Capolista:  CACCOLA F.C.
39502 Capolista:  CACCOLA F.C.
39503 Capolista:  CACCOLA F.C.
39504 Capolista:  CACCOLA F.C.
39505 Capolista:  CACCOLA F.C.
39506 Capolista:  CACCOLA F.C.
39507 Capolista:  CACCOLA F.C.
39508 Capolista:  CACCOLA F.C.
39509 Capolista:  CACCOLA F.C.
39510 Capolista:  Capitan Martella
39511 Capolista:  Capitan Martella
39512 Capolista:  CACCOLA F.C.
39513 Capolista:  CACCOLA F.C.
39514 Capolista:  CACCOLA F.C.
39515 Capolista:  CACCOLA F.C.
39516 Capolista:  Capitan Martella
39517 Capolista:  Capitan Martella
39518 Capolista:  CACCOLA F.C.
39519 Capolista:  Capitan Martella
39520 Capolista:  CACCOLA F.C.
39521 Capolista

39753 Capolista:  CACCOLA F.C.
39754 Capolista:  CACCOLA F.C.
39755 Capolista:  CACCOLA F.C.
39756 Capolista:  Paris FC
39757 Capolista:  Paris FC
39758 Capolista:  Paris FC
39759 Capolista:  Paris FC
39760 Capolista:  Paris FC
39761 Capolista:  CACCOLA F.C.
39762 Capolista:  Capitan Martella
39763 Capolista:  Paris FC
39764 Capolista:  Paris FC
39765 Capolista:  Paris FC
39766 Capolista:  Paris FC
39767 Capolista:  Paris FC
39768 Capolista:  Paris FC
39769 Capolista:  Capitan Martella
39770 Capolista:  Capitan Martella
39771 Capolista:  Capitan Martella
39772 Capolista:  CACCOLA F.C.
39773 Capolista:  CACCOLA F.C.
39774 Capolista:  CACCOLA F.C.
39775 Capolista:  Capitan Martella
39776 Capolista:  Paris FC
39777 Capolista:  Capitan Martella
39778 Capolista:  CACCOLA F.C.
39779 Capolista:  CACCOLA F.C.
39780 Capolista:  CACCOLA F.C.
39781 Capolista:  CACCOLA F.C.
39782 Capolista:  Paris FC
39783 Capolista:  Paris FC
39784 Capolista:  CACCOLA F.C.
39785 Capolista:  CACCOLA F.C.
39786 Cap

40026 Capolista:  CACCOLA F.C.
40027 Capolista:  Paris FC
40028 Capolista:  Paris FC
40029 Capolista:  CACCOLA F.C.
40030 Capolista:  SamPDORRR FC
40031 Capolista:  Paris FC
40032 Capolista:  Paris FC
40033 Capolista:  Paris FC
40034 Capolista:  Paris FC
40035 Capolista:  Paris FC
40036 Capolista:  Paris FC
40037 Capolista:  Paris FC
40038 Capolista:  Capitan Martella
40039 Capolista:  Capitan Martella
40040 Capolista:  Paris FC
40041 Capolista:  Paris FC
40042 Capolista:  Paris FC
40043 Capolista:  CACCOLA F.C.
40044 Capolista:  Paris FC
40045 Capolista:  Paris FC
40046 Capolista:  Paris FC
40047 Capolista:  Paris FC
40048 Capolista:  Paris FC
40049 Capolista:  CACCOLA F.C.
40050 Capolista:  Capitan Martella
40051 Capolista:  Capitan Martella
40052 Capolista:  Paris FC
40053 Capolista:  Paris FC
40054 Capolista:  Paris FC
40055 Capolista:  Paris FC
40056 Capolista:  CACCOLA F.C.
40057 Capolista:  Capitan Martella
40058 Capolista:  CACCOLA F.C.
40059 Capolista:  Paris FC
40060 Capolist

40291 Capolista:  CACCOLA F.C.
40292 Capolista:  CACCOLA F.C.
40293 Capolista:  CACCOLA F.C.
40294 Capolista:  CACCOLA F.C.
40295 Capolista:  CACCOLA F.C.
40296 Capolista:  CACCOLA F.C.
40297 Capolista:  CACCOLA F.C.
40298 Capolista:  Paris FC
40299 Capolista:  CACCOLA F.C.
40300 Capolista:  CACCOLA F.C.
40301 Capolista:  CACCOLA F.C.
40302 Capolista:  Capitan Martella
40303 Capolista:  CACCOLA F.C.
40304 Capolista:  Paris FC
40305 Capolista:  CACCOLA F.C.
40306 Capolista:  CACCOLA F.C.
40307 Capolista:  Capitan Martella
40308 Capolista:  Capitan Martella
40309 Capolista:  CACCOLA F.C.
40310 Capolista:  CACCOLA F.C.
40311 Capolista:  CACCOLA F.C.
40312 Capolista:  CACCOLA F.C.
40313 Capolista:  CACCOLA F.C.
40314 Capolista:  CACCOLA F.C.
40315 Capolista:  CACCOLA F.C.
40316 Capolista:  CACCOLA F.C.
40317 Capolista:  CACCOLA F.C.
40318 Capolista:  CACCOLA F.C.
40319 Capolista:  CACCOLA F.C.


In [78]:
df_persquadra["Percent"] = np.zeros(8)

In [98]:
pv = {s : 0 for s in squadre}
MediaPunti = {s : 0 for s in squadre}
i=0
for classifica in classifiche:
    pv[classifica.index[0]] +=1/np.math.factorial(8)
    for idx in classifica.index:
        MediaPunti[idx] += classifica[idx]/ np.math.factorial(8)
    print(i)
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3208
3209


5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257


7287
7288
7289
7290
7291
7292
7293
7294
7295
7296
7297
7298
7299
7300
7301
7302
7303
7304
7305
7306
7307
7308
7309
7310
7311
7312
7313
7314
7315
7316
7317
7318
7319
7320
7321
7322
7323
7324
7325
7326
7327
7328
7329
7330
7331
7332
7333
7334
7335
7336
7337
7338
7339
7340
7341
7342
7343
7344
7345
7346
7347
7348
7349
7350
7351
7352
7353
7354
7355
7356
7357
7358
7359
7360
7361
7362
7363
7364
7365
7366
7367
7368
7369
7370
7371
7372
7373
7374
7375
7376
7377
7378
7379
7380
7381
7382
7383
7384
7385
7386
7387
7388
7389
7390
7391
7392
7393
7394
7395
7396
7397
7398
7399
7400
7401
7402
7403
7404
7405
7406
7407
7408
7409
7410
7411
7412
7413
7414
7415
7416
7417
7418
7419
7420
7421
7422
7423
7424
7425
7426
7427
7428
7429
7430
7431
7432
7433
7434
7435
7436
7437
7438
7439
7440
7441
7442
7443
7444
7445
7446
7447
7448
7449
7450
7451
7452
7453
7454
7455
7456
7457
7458
7459
7460
7461
7462
7463
7464
7465
7466
7467
7468
7469
7470
7471
7472
7473
7474
7475
7476
7477
7478
7479
7480
7481
7482
7483
7484
7485
7486


9936
9937
9938
9939
9940
9941
9942
9943
9944
9945
9946
9947
9948
9949
9950
9951
9952
9953
9954
9955
9956
9957
9958
9959
9960
9961
9962
9963
9964
9965
9966
9967
9968
9969
9970
9971
9972
9973
9974
9975
9976
9977
9978
9979
9980
9981
9982
9983
9984
9985
9986
9987
9988
9989
9990
9991
9992
9993
9994
9995
9996
9997
9998
9999
10000
10001
10002
10003
10004
10005
10006
10007
10008
10009
10010
10011
10012
10013
10014
10015
10016
10017
10018
10019
10020
10021
10022
10023
10024
10025
10026
10027
10028
10029
10030
10031
10032
10033
10034
10035
10036
10037
10038
10039
10040
10041
10042
10043
10044
10045
10046
10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10

...

30418
30419
30420
30421
30422
30423
30424
30425
30426
30427
30428
30429
30430
30431
30432
30433
30434
30435
30436
30437
30438
30439
30440
30441
30442
30443
30444
30445
30446
30447
30448
30449
30450
30451
30452
30453
30454
30455
30456
30457
30458
30459
30460
30461
30462
30463
30464
30465
30466
30467
30468
30469
30470
30471
30472
30473
30474
30475
30476
30477
30478
30479
30480
30481
30482
30483
30484
30485
30486
30487
30488
30489
30490
30491
30492
30493
30494
30495
30496
30497
30498
30499
30500
30501
30502
30503
30504
30505
30506
30507
30508
30509
30510
30511
30512
30513
30514
30515
30516
30517
30518
30519
30520
30521
30522
30523
30524
30525
30526
30527
30528
30529
30530
30531
30532
30533
30534
30535
30536
30537
30538
30539
30540
30541
30542
30543
30544
30545
30546
30547
30548
30549
30550
30551
30552
30553
30554
30555
30556
30557
30558
30559
30560
30561
30562
30563
30564
30565
30566
30567
30568
30569
30570
30571
30572
30573
30574
30575
30576
30577
30578
30579
30580
30581
30582
30583
3058

32605
32606
32607
32608
32609
32610
32611
32612
32613
32614
32615
32616
32617
32618
32619
32620
32621
32622
32623
32624
32625
32626
32627
32628
32629
32630
32631
32632
32633
32634
32635
32636
32637
32638
32639
32640
32641
32642
32643
32644
32645
32646
32647
32648
32649
32650
32651
32652
32653
32654
32655
32656
32657
32658
32659
32660
32661
32662
32663
32664
32665
32666
32667
32668
32669
32670
32671
32672
32673
32674
32675
32676
32677
32678
32679
32680
32681
32682
32683
32684
32685
32686
32687
32688
32689
32690
32691
32692
32693
32694
32695
32696
32697
32698
32699
32700
32701
32702
32703
32704
32705
32706
32707
32708
32709
32710
32711
32712
32713
32714
32715
32716
32717
32718
32719
32720
32721
32722
32723
32724
32725
32726
32727
32728
32729
32730
32731
32732
32733
32734
32735
32736
32737
32738
32739
32740
32741
32742
32743
32744
32745
32746
32747
32748
32749
32750
32751
32752
32753
32754
32755
32756
32757
32758
32759
32760
32761
32762
32763
32764
32765
32766
32767
32768
32769
32770
3277

34895
34896
34897
34898
34899
34900
34901
34902
34903
34904
34905
34906
34907
34908
34909
34910
34911
34912
34913
34914
34915
34916
34917
34918
34919
34920
34921
34922
34923
34924
34925
34926
34927
34928
34929
34930
34931
34932
34933
34934
34935
34936
34937
34938
34939
34940
34941
34942
34943
34944
34945
34946
34947
34948
34949
34950
34951
34952
34953
34954
34955
34956
34957
34958
34959
34960
34961
34962
34963
34964
34965
34966
34967
34968
34969
34970
34971
34972
34973
34974
34975
34976
34977
34978
34979
34980
34981
34982
34983
34984
34985
34986
34987
34988
34989
34990
34991
34992
34993
34994
34995
34996
34997
34998
34999
35000
35001
35002
35003
35004
35005
35006
35007
35008
35009
35010
35011
35012
35013
35014
35015
35016
35017
35018
35019
35020
35021
35022
35023
35024
35025
35026
35027
35028
35029
35030
35031
35032
35033
35034
35035
35036
35037
35038
35039
35040
35041
35042
35043
35044
35045
35046
35047
35048
35049
35050
35051
35052
35053
35054
35055
35056
35057
35058
35059
35060
3506

37181
37182
37183
37184
37185
37186
37187
37188
37189
37190
37191
37192
37193
37194
37195
37196
37197
37198
37199
37200
37201
37202
37203
37204
37205
37206
37207
37208
37209
37210
37211
37212
37213
37214
37215
37216
37217
37218
37219
37220
37221
37222
37223
37224
37225
37226
37227
37228
37229
37230
37231
37232
37233
37234
37235
37236
37237
37238
37239
37240
37241
37242
37243
37244
37245
37246
37247
37248
37249
37250
37251
37252
37253
37254
37255
37256
37257
37258
37259
37260
37261
37262
37263
37264
37265
37266
37267
37268
37269
37270
37271
37272
37273
37274
37275
37276
37277
37278
37279
37280
37281
37282
37283
37284
37285
37286
37287
37288
37289
37290
37291
37292
37293
37294
37295
37296
37297
37298
37299
37300
37301
37302
37303
37304
37305
37306
37307
37308
37309
37310
37311
37312
37313
37314
37315
37316
37317
37318
37319
37320
37321
37322
37323
37324
37325
37326
37327
37328
37329
37330
37331
37332
37333
37334
37335
37336
37337
37338
37339
37340
37341
37342
37343
37344
37345
37346
3734

39441
39442
39443
39444
39445
39446
39447
39448
39449
39450
39451
39452
39453
39454
39455
39456
39457
39458
39459
39460
39461
39462
39463
39464
39465
39466
39467
39468
39469
39470
39471
39472
39473
39474
39475
39476
39477
39478
39479
39480
39481
39482
39483
39484
39485
39486
39487
39488
39489
39490
39491
39492
39493
39494
39495
39496
39497
39498
39499
39500
39501
39502
39503
39504
39505
39506
39507
39508
39509
39510
39511
39512
39513
39514
39515
39516
39517
39518
39519
39520
39521
39522
39523
39524
39525
39526
39527
39528
39529
39530
39531
39532
39533
39534
39535
39536
39537
39538
39539
39540
39541
39542
39543
39544
39545
39546
39547
39548
39549
39550
39551
39552
39553
39554
39555
39556
39557
39558
39559
39560
39561
39562
39563
39564
39565
39566
39567
39568
39569
39570
39571
39572
39573
39574
39575
39576
39577
39578
39579
39580
39581
39582
39583
39584
39585
39586
39587
39588
39589
39590
39591
39592
39593
39594
39595
39596
39597
39598
39599
39600
39601
39602
39603
39604
39605
39606
3960

In [109]:
ma = []
i=0
for classifica in classifiche:
    if classifica['CACCOLA F.C.'] >25 and classifica['Capitan Martella'] <16 :
        ma.append(i)
    i +=1

## Analisi vera e propria

In [ ]:
g1,g2

In [ ]:
#ora che ho il dataset pronto, posso passare all'analisi
from itertools import permutations, product #permutations non serve ad un cazzo
from matplotlib import pyplot as plt #matplotlib
%matplotlib inline
import seaborn as sns #seaborn
plt.rcParams['figure.figsize'] = 15,5 #grandezza grafico

In [ ]:
lista_confronti=[]
for i in data.columns: #per ogni colonna (ogni giornata)...
    i = list(product(data[i], data[i])) #... paragonami ogni numero, quindi 64 (8*8) coppie per ogni giornata
    lista_confronti.append(i)

In [ ]:
giornate = len(lista_confronti) #quante partite abbiamo giorcato
matrici88_giornate = [] #matrici 8*8
for i in range(giornate): #cioè per ogni giornata
    for elementi in lista_confronti[i]: 
        elementi = elementi[0] - elementi[1] #goal fatto - subito, cioè le coppie che ho generato in lista_confronti (sopra)
        if elementi > 0: #vittoria
            elementi = 3
        elif elementi < 0: #sconfitta
            elementi = 0
        else:  #pareggio
            elementi = 1
        matrici88_giornate.append(elementi)
matrici88_giornate = np.array(matrici88_giornate).reshape((giornate, 8, 8)) 
#organizza la lista che hai appena creato nel seguente modo:
    #ci sono tante matrici 8*8 quante sono le giornate
matrici88_giornate #ogni matrice è una giornata
#(sulle righe di ogni matrice abbiamo la performance degli otto giocatori)

In [ ]:
#creo questa matrice per togliere il punto che ad ogni giornata veniva precendentemente assegnato di default 
    #(questo accadeva perché tutti giocavano contro se stessi)
validation_matrix = np.ones([giornate,8,8])
d3, row, col = validation_matrix.shape
for k in range(d3):
    for i in range(row):
        for j in range(col):
            if i == j: 
                validation_matrix[k,i,j] = 0

matrici88_giornate = matrici88_giornate * validation_matrix

## I punti in ogni giornata

In [ ]:
#non ricordo come funziona la np.sum, con axis=2, funziona solo per 3D array, era nella lezione03_03 del corso LinkedIn
punti_giornata = matrici88_giornate.sum(axis=2)  
punti_giornata #ora ogni riga rappresenta una giornata, ogni colonna un giocatore, ma a me serve la trasposta

In [ ]:
punti_giornata = punti_giornata.T #ora ogni matrice rappresenta la performance di ogni giocatore nelle giornate
punti_giornata = pd.DataFrame(punti_giornata) 
punti_giornata.set_index(data.index, inplace=True)
punti_giornata.columns = data.columns

In [ ]:
#setto i nomi delle variabili, ora sono uguali ai nomi delle variabili del dataset originale
punti_giornata.columns = data.columns #ma voglio cambiare la parola 'goal' che non va bene
nuovi_nomi = []
for elementi in punti_giornata.columns:
    elementi = elementi.replace('goal','punti') #sostituisci la parola 'goal' con la parola 'punti'
    nuovi_nomi.append(elementi)
punti_giornata.columns = nuovi_nomi
punti_giornata = punti_giornata.astype(int) #trasformiamo i valori del dataframe in int
punti_giornata

## Heatmap

In [ ]:
plt.rcParams['figure.figsize'] = 15,7
sns.heatmap(punti_giornata.T, cmap="PiYG", xticklabels=punti_giornata.index, yticklabels=False); 
#at the bottom trovi le ultime giornate, in alto le prime 

## Classifica Finale

In [ ]:
#non sono sicuro che devo necessariamente trasformare il dataframe in np.matrix/array per poter sommare orizzontalmente
    #(axis=0 somma verticale)
classifica_finale = np.asmatrix(punti_giornata).sum(axis=1)

In [ ]:
data['classifica_finale'] = classifica_finale #aggiungo la classifica finale al dataset 
data.sort_values(["classifica_finale"], ascending=False, inplace=True) #ordinare dal più alto al più basso
classifica_finale = data['classifica_finale'] #per tenere solo la classifica (creo un altro dataset)
classifica_finale

In [ ]:
(classifica_finale/7).round(1)

## Barchart

In [ ]:
import warnings
warnings.filterwarnings('ignore')

sns.set(style="whitegrid")
sns.barplot(x=data.index, y=data.classifica_finale);